In [1]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('../carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


import carla

import time
import math
import hydra
import logging
import argparse


from numpy import random
from omegaconf import DictConfig, OmegaConf, ValidationError

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.pgm import PGM
from utils.weather import Weather
from core.vehicle_manager import VehicleManager
from core.pedestrian_manager import PedestrianManager
from utils.common import get_actor_blueprints, hour_to_altitude

from hydra.core.config_store import ConfigStore
from schemas.pgm_schema import PGMModel
from schemas.weather_schema import WeatherSchema
from schemas.config_schema import ConfigSchema

c:\Users\aadim\.conda\envs\carla-pgm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cs = ConfigStore.instance()
cs.store(group="weather", name="base_weather_model", node=WeatherSchema)
cs.store(group="model", name="base_pgm_model", node=PGMModel)
cs.store(name="base_config_model", node=ConfigSchema)

with hydra.initialize(version_base=None, config_path="../conf"):
    cfg = hydra.compose(
        config_name="config", 
        overrides=[
            "variables=trained",
            "weather=rain",
            "weather.hour=9"
            ]
        )

pgm = PGM(cfg=cfg, file_path="eCARLA_PGM.xml")
pgm.print_cpd('Traffic')

# pgm = PGM(file_path="eCARLA_PGM.xml")
# pgm.model.to_daft().render()

+-----------------+-----+---------------------+
| Fog             | ... | Fog(NO_FOG)         |
+-----------------+-----+---------------------+
| Pedestrians     | ... | Pedestrians(MEDIUM) |
+-----------------+-----+---------------------+
| Time            | ... | Time(NIGHT)         |
+-----------------+-----+---------------------+
| Traffic(HEAVY)  | ... | 0.3333333333333333  |
+-----------------+-----+---------------------+
| Traffic(LOW)    | ... | 0.3333333333333333  |
+-----------------+-----+---------------------+
| Traffic(MEDIUM) | ... | 0.3333333333333333  |
+-----------------+-----+---------------------+


In [3]:
pgm_states = pgm.get_states()
pgm_variables = pgm.get_variables()

In [4]:
# for s in pgm_states.Time:
#     print(s.name)

In [5]:
pgm.predict_state([pgm_variables.Traffic], evidence={
    pgm_variables.Fog: pgm_states.Fog.HEAVY.name
})[pgm_variables.Traffic]

Eliminating: Time: 100%|██████████| 4/4 [00:00<00:00, 444.36it/s]


'HEAVY'

In [6]:
def time_of_day(hour):
    if hour >= 2 and hour <= 6:
        return 'EARLY_MORNING'
    elif hour > 6 and hour <= 9:
        return 'MORNING'
    elif hour > 9 and hour <= 12:
        return 'LATE_MORNING'
    elif hour > 12 and hour <= 17:
        return 'AFTERNOON'
    elif hour > 17 and hour <= 19:
        return 'EARLY_EVENING'
    elif hour > 19 and hour <= 21:
        return 'EVENING'
    elif hour > 21 and hour <= 23:
        return 'LATE_EVENING'
    else:
        return 'NIGHT'

In [7]:
for i in range(24):
    print(f"Hour {i} : {hour_to_altitude(i)}: {time_of_day(i)}")

Hour 0 : -90: NIGHT
Hour 1 : -75: NIGHT
Hour 2 : -60: EARLY_MORNING
Hour 3 : -45: EARLY_MORNING
Hour 4 : -30: EARLY_MORNING
Hour 5 : -15: EARLY_MORNING
Hour 6 : 0: EARLY_MORNING
Hour 7 : 15: MORNING
Hour 8 : 30: MORNING
Hour 9 : 45: MORNING
Hour 10 : 60: LATE_MORNING
Hour 11 : 75: LATE_MORNING
Hour 12 : 90: LATE_MORNING
Hour 13 : 75: AFTERNOON
Hour 14 : 60: AFTERNOON
Hour 15 : 45: AFTERNOON
Hour 16 : 30: AFTERNOON
Hour 17 : 15: AFTERNOON
Hour 18 : 0: EARLY_EVENING
Hour 19 : -15: EARLY_EVENING
Hour 20 : -30: EVENING
Hour 21 : -45: EVENING
Hour 22 : -60: LATE_EVENING
Hour 23 : -75: LATE_EVENING


In [8]:
# pgm.print_cpd('Speed')

In [9]:
def get_state_from_config(value, states):
     for s in states:
        if (value <= s.value):
            return s.name

In [23]:
def main(cfg: DictConfig):
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
    
    client = carla.Client(cfg.host, cfg.port)
    client.set_timeout(10.0)

    pgm = PGM(cfg, file_path="eCARLA_PGM.xml")
    pgm_variables = pgm.get_variables()
    pgm_states = pgm.get_states()

    # get weather and time state from configuration files
    rain_state = get_state_from_config(cfg.weather.rain, pgm_states.Rain)
    print(f"Rain state: {rain_state}")

    fog_state = get_state_from_config(cfg.weather.fog, pgm_states.Fog)
    print(f"Fog state: {fog_state}")

    clouds_state = get_state_from_config(cfg.weather.cloudiness, pgm_states.Clouds)
    print(f"Cloud state: {clouds_state}")

    time_state = time_of_day(cfg.weather.hour)
    print(f"Time of day: {time_state}")

    pedestrians_state = pgm.predict_state(
        [pgm_variables.Pedestrians], 
        evidence={
            pgm_variables.Rain: rain_state,
            pgm_variables.Time: time_state
        }
    )[pgm_variables.Pedestrians]
    print(f"No. of Pedestrians: {pedestrians_state}")

    traffic_state = pgm.predict_state(
        [pgm_variables.Traffic], 
        evidence={
            pgm_variables.Fog: fog_state,
            pgm_variables.Pedestrians: pedestrians_state,
            pgm_variables.Time: time_state
        }
    )[pgm_variables.Traffic]
    print(f"No. of Vehicles: {traffic_state}")

    speed_state = pgm.predict_state(
        [pgm_variables.Speed], 
        evidence={
            pgm_variables.Fog: fog_state,
            pgm_variables.Pedestrians: pedestrians_state,
            pgm_variables.Time: time_state,
            pgm_variables.Traffic: traffic_state
        }
    )[pgm_variables.Speed]
    print(f"Speed of Vehicles: {speed_state}")

    rng = random.default_rng()

    num_of_vehicles = pgm_states.Traffic[traffic_state].value
    num_of_vehicles = rng.poisson(num_of_vehicles)
    print(f"Spawning {num_of_vehicles} vehicles.")

    speed = pgm_states.Speed[speed_state].value
    speed = rng.poisson(speed)
    print(f"Maximum speed is {100 - speed}% of the speed limit.")

    num_of_pedestrians = pgm_states.Pedestrians[pedestrians_state].value
    num_of_pedestrians = rng.poisson(num_of_pedestrians)
    print(f"Spawning {num_of_pedestrians} pedestrians.")

    vcfg = hydra.compose(config_name="vehicle_manager.yaml")
    tm = VehicleManager(client=client, cfg=vcfg, asynch=cfg.asynch)

    speed_history = list()

    pcfg = hydra.compose(config_name="pedestrian_manager.yaml")
    # print(pcfg)
    pm = PedestrianManager(client=client, cfg=pcfg, asynch=cfg.asynch)
    try:
        world = client.get_world()

        all_vehicles = tm.spawn_vehicles(number_of_vehicles=num_of_vehicles)
        pm.spawn_walkers(number_of_walkers=num_of_pedestrians, percentagePedestriansRunning=pcfg.running_percentage, percentagePedestriansCrossing=pcfg.crossing_percentage)

        # Dynamic Weather
        speed_factor = 1.0
        update_freq = 0.5 / speed_factor

        weather = Weather(world.get_weather())
        weather.setWeather(cfg.weather)
        world.set_weather(weather.weather)

        elapsed_time = 0.0

        tm.update_global_speed(speed)

        synchronous_master = world.get_settings().synchronous_mode
        while True:
            if not cfg.asynch and synchronous_master:
                world.tick()
            else:
                timestamp = world.wait_for_tick(seconds=30.0).timestamp
                elapsed_time += timestamp.delta_seconds
            v = world.get_actor(all_vehicles[0]).get_velocity()
            current_speed = 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
            speed_history.append(current_speed)
            # print('Speed:   % 15.0f km/h' % (current_speed))
    finally:

        tm.destroy()
        pm.destroy()
        time.sleep(0.5)
        return speed_history

In [25]:
try:
    cs = ConfigStore.instance()
    cs.store(group="weather", name="base_weather_model", node=WeatherSchema)
    cs.store(group="model", name="base_pgm_model", node=PGMModel)
    cs.store(name="base_config_model", node=ConfigSchema)
    with hydra.initialize(version_base=None, config_path="../conf"):
        cfg = hydra.compose(
            config_name="config", 
            overrides=[
                "weather=clear",
                "weather.hour=8",
                "variables=trained"
                ])
        print(cfg.weather)
        clear_speeds = main(cfg)
except KeyboardInterrupt:
    pass
finally:
    print('\nDone.')

{'cloudiness': 10.0, 'rain': 0.0, 'puddles': 0.0, 'wind': 5.0, 'fog': 0.0, 'wetness': 0.0, 'hour': 8}
Rain state: NO_RAIN
Fog state: NO_FOG
Rain state: NO_CLOUDS
Time of day: MORNING


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


No. of Pedestrians: MEDIUM


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


No. of Vehicles: MEDIUM


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

Speed of Vehicles: LOW
Spawning 20 vehicles.
50
40
Maximum speed is 60% of the speed limit.
Spawning 7 pedestrians.



ERROR: Spawn failed because of collision at spawn position
ERROR: Spawn failed because of collision at spawn position



Added 20 vehicles
Spawned 5 walkers.

Destroying 20 vehicles
Destroying 5 walkers

Done.
